In [1]:
import numpy as np

def generate_data(num_rows=200, num_columns=15, n_f=8, random_seed=0):
    # Set random seed for reproducibility
    np.random.seed(random_seed)

    # Generate Low-Rank Data
    D = np.diag([1e5, 1e3, 1e2, 10, 0.001, 0.00001, 0.0000001, 0, 0, 0, 0, 0, 0, 0, 0])
    V, _ = np.linalg.qr(np.random.rand(num_rows, num_columns))
    U, _ = np.linalg.qr(np.random.rand(num_rows, num_columns))
    test = U @ D @ V.T

    # Generate a basis matrix (example basis)
    f_basis = U[:, :n_f]

    return f_basis, test

def main_algorithm(f_basis, n_f=8, n_required_samples=None):
    # Step 1: QR Factorization - Obtain orthogonal basis Q from f_basis

    if n_required_samples is None:
        n_required_samples = n_f

    Q, R = np.linalg.qr(f_basis)
    Q_col_len, _ = Q.shape

    # Initialization - Select the initial index with the maximum value in the first column of Q
    i_star = np.argmax(np.abs(Q[:, 0]))
    Z_indices = [i_star]

    # Construct initial Z_matrix
    Z_matrix = np.zeros((Q_col_len, 1))
    Z_matrix[i_star, 0] = 1
    E_ = np.eye(n_f)

    # Iteratively select features
    for j in range(1, n_f):
        # Construct E_j matrix (first j standard basis vectors)
        E_j = E_[:, :j]

        # Construct A = Z^T * Q * E_j
        A = Z_matrix.T @ Q @ E_j  # Shape (len(Z_indices), j)

        # Construct c = Z^T * Q * e_{j+1}
        e_j_plus1 = np.zeros(n_f)
        e_j_plus1[j] = 1
        c = Z_matrix.T @ Q @ e_j_plus1  # Shape (len(Z_indices),)

        # Compute g
        AtA_inv = np.linalg.inv(A.T @ A)  # Inverse of A^T * A
        g = AtA_inv @ (A.T @ c)

        # Precompute norms of columns of A squared
        norm_A_columns_squared = np.sum(A ** 2, axis=0)  # Shape (j,)

        # Initialize variables to keep track of the maximum objective
        istar_max_obj = -np.inf
        i_star_candidate = -1

        # Iterate over candidate indices (not in Z_indices)
        candidates = [i for i in range(Q_col_len) if i not in Z_indices]
        for i in candidates:
            # Construct e_i vector
            e_i = np.zeros((Q_col_len, 1))
            e_i[i] = 1

            # Compute r = e_i^T * Q * E_j
            r_T = e_i.T @ Q @ E_j  # Shape (1, j)
            r = r_T.T  # Transpose to match the shape (j, 1)
            b = AtA_inv @ r  # Shape (j,)

            # Compute gamma = e_i^T * Q * e_{j+1}
            gamma = Q[i, j]

            # Compute alpha
            cTA = c.T @ A
            gamma_rT = gamma * r_T
            numerator_alpha = cTA + gamma_rT
            denominator_alpha = 1 + r_T @ b
            correction_matrix = np.eye(j) - np.outer(b, r_T) / denominator_alpha
            g_plus_gamma_b = g + gamma * b.flatten()
            alpha = numerator_alpha.flatten() @ correction_matrix @ g_plus_gamma_b

            # Calculate objective to find the best candidate
            cTc = c.T @ c
            numerator1_istar = 1 + r_T @ b
            denominator1_istar = np.prod(norm_A_columns_squared + r ** 2)
            numerator2_istar = cTc + gamma ** 2 - alpha
            denominator2_istar = cTc + gamma ** 2

            istar_obj = (numerator1_istar / denominator1_istar) * (numerator2_istar / denominator2_istar)

            if istar_obj > istar_max_obj:
                istar_max_obj = istar_obj
                i_star_candidate = i

        # Update Z_indices with the index that maximizes the objective
        Z_indices.append(i_star_candidate)

        # Update Z_matrix by adding a new column for the new index
        Z_new_column = np.zeros((Q_col_len, 1))
        Z_new_column[i_star_candidate, 0] = 1
        Z_matrix = np.hstack((Z_matrix, Z_new_column))

        
    # Perform oversampling if needed
    if n_required_samples > n_f:
        num_additional_samples = n_required_samples - len(Z_indices)
        for _ in range(num_additional_samples):
            A = Z_matrix.T @ Q
            AtA_inv = np.linalg.pinv(A.T @ A)
            istar_max_obj = -np.inf
            i_star_candidate = -1

            # Iterate over candidate indices for oversampling
            candidates = [i for i in range(Q_col_len) if i not in Z_indices]
            for i in candidates:
                r = Q[i, :]
                norm_A_columns_squared = np.sum(A ** 2, axis=0)
                numerator_istar = 1 + r.T @ AtA_inv @ r
                denominator_istar = np.prod(norm_A_columns_squared + r ** 2)
                istar_obj = numerator_istar / denominator_istar

                # Update the best candidate if the current objective is better
                if istar_obj > istar_max_obj:
                    istar_max_obj = istar_obj
                    i_star_candidate = i

            # Update Z_indices and Z_matrix with the selected candidate
            Z_indices.append(i_star_candidate)
            Z_new_column = np.zeros((Q_col_len, 1))
            Z_new_column[i_star_candidate, 0] = 1
            Z_matrix = np.hstack((Z_matrix, Z_new_column))

    return Z_indices, Z_matrix

def reconstruct_basis(f_basis, Z_indices, Z_matrix, test):
    # Extract sampled rows from the original basis
    f_sampled_row = f_basis[Z_indices, :]

    # Compute the pseudoinverse of the sampled basis
    f_basis_sampled_inv = np.linalg.pinv(f_sampled_row)

    # Reconstruct the basis
    s_mat = f_basis @ f_basis_sampled_inv @ Z_matrix.T
    test_rec = s_mat @ test

    # Calculate the reconstruction error
    reconstruction_error = np.linalg.norm(test_rec - test, 'fro')

    return f_sampled_row, f_basis_sampled_inv, reconstruction_error

In [2]:

# Run the main algorithm
if __name__ == "__main__":
    f_basis, test = generate_data(n_f=10)
    n_required_samples = 20
    Z_indices, Z_matrix = main_algorithm(f_basis,n_f=10,n_required_samples=n_required_samples)
    f_sampled_row, f_basis_sampled_inv, reconstruction_error = reconstruct_basis(f_basis, Z_indices, Z_matrix, test)

    print("Selected indices Z:", Z_indices)
    # print("\nSampled Rows (f_sampled_row):")
    # print(f_sampled_row)
    # print("\nPseudoinverse of Sampled Basis (f_basis_sampled_inv):")
    # print(f_basis_sampled_inv)
    print("\nReconstruction Error:", reconstruction_error)

Selected indices Z: [139, 160, 176, 103, 133, 156, 48, 66, 14, 102, 175, 96, 132, 111, 86, 123, 10, 91, 2, 4]

Reconstruction Error: 5.7476982948734546e-11


In [3]:
# def sopt_red(self,f_basis, num_f_basis_vectors_used):

#     # Step 1: QR Factorization - Obtain orthogonal basis Q from f_basis
#             # Initialize variables
#     n_f = min(num_f_basis_vectors_used, f_basis.shape[1])

#     f_basis = f_basis[:,:n_f]

#     Q, R = np.linalg.qr(f_basis)
#     Q_col_len, _ = Q.shape

#     # Initialization - Select the initial index with the maximum value in the first column of Q
#     i_star = np.argmax(np.abs(Q[:, 0]))
#     Z_indices = [i_star]

#     # Construct initial Z_matrix
#     Z_matrix = np.zeros((Q_col_len, 1))
#     Z_matrix[i_star, 0] = 1
#     E_ = np.eye(n_f)

#     # Iteratively select features
#     for j in range(1, n_f):
#         # Construct E_j matrix (first j standard basis vectors)
#         E_j = E_[:, :j]

#         # Construct A = Z^T * Q * E_j
#         A = Z_matrix.T @ Q @ E_j  # Shape (len(Z_indices), j)

#         # Construct c = Z^T * Q * e_{j+1}
#         e_j_plus1 = np.zeros(n_f)
#         e_j_plus1[j] = 1
#         c = Z_matrix.T @ Q @ e_j_plus1  # Shape (len(Z_indices),)

#         # Compute g
#         AtA_inv = np.linalg.inv(A.T @ A)  # Inverse of A^T * A
#         g = AtA_inv @ (A.T @ c)

#         # Precompute norms of columns of A squared
#         norm_A_columns_squared = np.sum(A ** 2, axis=0)  # Shape (j,)

#         # Initialize variables to keep track of the maximum objective
#         istar_max_obj = -np.inf
#         i_star_candidate = -1

#         # Iterate over candidate indices (not in Z_indices)
#         candidates = [i for i in range(Q_col_len) if i not in Z_indices]

#         for i in candidates:
#             # Construct e_i vector
#             e_i = np.zeros((Q_col_len, 1))
#             e_i[i] = 1

#             # Compute r = e_i^T * Q * E_j
#             r_T = e_i.T @ Q @ E_j  # Shape (1, j)
#             r = r_T.T  # Transpose to match the shape (j, 1)
#             b = AtA_inv @ r  # Shape (j,)

#             # Compute gamma = e_i^T * Q * e_{j+1}
#             gamma = Q[i, j]

#             # Compute alpha
#             cTA = c.T @ A
#             gamma_rT = gamma * r_T
#             numerator_alpha = cTA + gamma_rT
#             denominator_alpha = 1 + r_T @ b
#             correction_matrix = np.eye(j) - np.outer(b, r_T) / denominator_alpha
#             g_plus_gamma_b = g + gamma * b.flatten()
#             alpha = numerator_alpha.flatten() @ correction_matrix @ g_plus_gamma_b

#             # Calculate objective to find the best candidate
#             cTc = c.T @ c
#             numerator1_istar = 1 + r_T @ b
#             denominator1_istar = np.prod(norm_A_columns_squared + r ** 2)
#             numerator2_istar = cTc + gamma ** 2 - alpha
#             denominator2_istar = cTc + gamma ** 2

#             istar_obj = (numerator1_istar / denominator1_istar) * (numerator2_istar / denominator2_istar)

#             if istar_obj > istar_max_obj:
#                 istar_max_obj = istar_obj
#                 i_star_candidate = i

#         # Update Z_indices with the index that maximizes the objective
#         Z_indices.append(i_star_candidate)

#         # Update Z_matrix by adding a new column for the new index
#         Z_new_column = np.zeros((Q_col_len, 1))
#         Z_new_column[i_star_candidate, 0] = 1
#         Z_matrix = np.hstack((Z_matrix, Z_new_column))

#     # Create a boolean vector indicating selected indices
#     Z_boolean = np.zeros(Q_col_len, dtype=bool)
#     Z_boolean[Z_indices] = True

#     f_sampled_row = f_basis[Z_boolean, :]
#     print(f"{Z_indices=}")
    
#     return f_sampled_row, Z_indices, Z_boolean

